참고 : https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 50 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=b9d4ffa47e3c343f0b0045e2d655412cd9f72b6c4b402e134e16b8001d5625c9
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 769 kB 5.3 MB/s 
     |████████████████████████████████| 3.0 MB 39.7 MB/s 
     |████████████████████████████████| 895 kB 28.8 MB/s 


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3mzaj6zq
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3mzaj6zq
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=b5aeaae95db959fc1542261ba82945ecf771ab4cd2af0efd663e9f22f0c14a9f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5bnrt20u/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
device = torch.device("cuda:0")

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/한국어_단발성_대화_데이터셋.xlsx')
# AIHUB '한국어 감정 정보가 포함된 단발성 대화 데이터셋' 다운로드
# https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-009

In [28]:
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [29]:
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [30]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [32]:
max_len = 64#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [33]:
BERTDataset(data_list, 0, 1, tok, max_len, True, False)

In [9]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [18]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [19]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [36]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5\

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

In [22]:
import os
os.chdir('/content/drive/MyDrive/')

model1 = torch.load('7emotions_model.pt') 
# https://drive.google.com/file/d/1a0p8attf7ubf9UZLiQfYlmyoA_lemPBP/view?usp=sharing
model1.load_state_dict(torch.load('7emotions_model_state_dict.pt'))  
# https://drive.google.com/file/d/191FecNZTkK_ibGbV6r0_ZY8r4nA8mb3u/view?usp=sharing
checkpoint = torch.load('7emotions_all.tar') 
# https://drive.google.com/file/d/1-6mHiVmJr9o229edxuh4RWULoJpkujTQ/view?usp=sharing
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [48]:
def predict(dataset_another):
    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람")
            elif np.argmax(logits) == 2:
                test_eval.append("분노")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 4:
                test_eval.append("중립")
            elif np.argmax(logits) == 5:
                test_eval.append("행복")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오")

        return test_eval

In [59]:
list = [['영화에 나오는 귀신이 너무 무섭네요'],['그게 사실이야? 대박'],['배고파서 화가 난다'],
                   ['그런 일이 있었다니 참 안타깝다'],['오늘은 비가 온다고 해요~'], ['대학교에 붙어서 기분이 너무 좋다'], ['수준 너무 떨어진다~']]

In [60]:
for i in list:
  i.append('0')
print(list)

[['영화에 나오는 귀신이 너무 무섭네요', '0'], ['그게 사실이야? 대박', '0'], ['배고파서 화가 난다', '0'], ['그런 일이 있었다니 참 안타깝다', '0'], ['오늘은 비가 온다고 해요~', '0'], ['대학교에 붙어서 기분이 너무 좋다', '0'], ['수준 너무 떨어진다~', '0']]


In [49]:
predict(list)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


['공포', '놀람', '분노', '슬픔', '행복', '행복', '혐오']

In [50]:
pip install kss

     |████████████████████████████████| 42.4 MB 69 kB/s 
     |████████████████████████████████| 170 kB 42.7 MB/s 
  Created wheel for kss: filename=kss-3.3.0-py3-none-any.whl size=42449194 sha256=c147499455e98608416777019669dbc1ee22875b94187196b1bde19f9089f5d8
  Stored in directory: /root/.cache/pip/wheels/0e/16/eb/7c53a9fa7971bf12ea368b4e1f265c3848135cd217fa87ad68
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=fab8b0968a0eb53f53a1e3d87d9287971d9e8dbd005bbc4869511671ccb90f8e
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built kss emoji


In [56]:
import kss
text = '오늘 먹은 저녁은 맛이 없었어요. 하지만 내일 아침은 맛있을거야. 분위기도 좋고 기분도 좋아! 진짜 무섭다 조심해야겠어. 별로인데... 진짜 세상에 말도 안되!'
sent_list = []
for sent in kss.split_sentences(text):
  sent_list.append([sent, '0']) 
print(sent_list)

[['오늘 먹은 저녁은 맛이 없었어요.', '0'], ['하지만 내일 아침은 맛있을거야.', '0'], ['분위기도 좋고 기분도 좋아!', '0'], ['진짜 무섭다 조심해야겠어.', '0'], ['별로인데...', '0'], ['진짜 세상에 말도 안되!', '0']]


In [61]:
result = predict(sent_list)
print(result)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


['슬픔', '행복', '행복', '공포', '슬픔', '놀람']


In [62]:
for j in sent_list:
  j.append(result.pop(0))
print(sent_list)

[['오늘 먹은 저녁은 맛이 없었어요.', '0', '슬픔'], ['하지만 내일 아침은 맛있을거야.', '0', '행복'], ['분위기도 좋고 기분도 좋아!', '0', '행복'], ['진짜 무섭다 조심해야겠어.', '0', '공포'], ['별로인데...', '0', '슬픔'], ['진짜 세상에 말도 안되!', '0', '놀람']]
